# Capstone Project Notebook - Week #5

## Final Project

## Evaluating Retail Lease Sites for Model Railroad Hobby Shop

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None) # required later
pd.set_option('display.max_rows', None) # required later

print('Libraries imported.')

Libraries imported.


### The retail lease data was loaded into my GitHub account as a csv

In [21]:
# scrape web page for table data 
df = pd.read_html('https://github.com/tdallimore/Coursera_Capstone/blob/master/capstone_project_week5.csv')

# assign only first table on web page to dataframe
df_data = df[0]
df_data.head()

,Unnamed: 0,Address,Latitude,Longitude,SquareFeet,LeaseRate,Walkable
0,NaN,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,1000,$35.00,No
1,NaN,"1748 Commercial Drive, Vancouver",49.268816,-123.069260,1325,$13.25,No
2,NaN,"1599 Venables St, Vancouver",49.276962,-123.072991,6600,$16.00,No
3,NaN,"430 Railway St, Vancouver",49.284616,-123.094533,8775,$18.74,No
4,NaN,"1951 Glen Dr, Vancouver",49.267358,-123.082534,534,$25.46,No


In [22]:
# drop 'Unnamed: 0' column with NaN
df_data.drop(['Unnamed: 0'], axis=1, inplace=True)
df_data.head()

,Address,Latitude,Longitude,SquareFeet,LeaseRate,Walkable
0,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,1000,$35.00,No
1,"1748 Commercial Drive, Vancouver",49.268816,-123.069260,1325,$13.25,No
2,"1599 Venables St, Vancouver",49.276962,-123.072991,6600,$16.00,No
3,"430 Railway St, Vancouver",49.284616,-123.094533,8775,$18.74,No
4,"1951 Glen Dr, Vancouver",49.267358,-123.082534,534,$25.46,No


### GeoData is now cleaned.  Next need to load the FourSquare, k-clustering and Mapping libraries

In [23]:
import json # library to handle JSON files

# install and import geopy for Foursquare API
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# install and import Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [42]:
# define map of Vancouver
vancouver_map = folium.Map(location=[49.280000, -123.046053], zoom_start=12, tiles='Stamen Terrain')

# add address markers to map
for lat, lng, label in zip(df_data['Latitude'], df_data['Longitude'], df_data['Address']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(vancouver_map)  

# show map
vancouver_map

In [32]:
# define Foursquare Credentials and Version

CLIENT_ID = 'U5L4NWGMXHTGERAFDMMSWIG3H1VHLQIV4EIAKKVWNAO5VPRW' # Foursquare ID
CLIENT_SECRET = 'WK234YY1I1THXQJO11OQH35LY4FLCJQ3Q1REU3E21JHOXW5Y' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: U5L4NWGMXHTGERAFDMMSWIG3H1VHLQIV4EIAKKVWNAO5VPRW
CLIENT_SECRET:WK234YY1I1THXQJO11OQH35LY4FLCJQ3Q1REU3E21JHOXW5Y


In [33]:
# get first address in the dataframe

df_data.loc[0, 'Address']

'2825 Grandview Hwy, Vancouver'

In [35]:
# get the address's latitude and longitude values

address_latitude = df_data.loc[0, 'Latitude'] # address latitude value
address_longitude = df_data.loc[0, 'Longitude'] # address longitude value

address_name = df_data.loc[0, 'Address'] # address name

print('Latitude and longitude values of {} are {}, {}.'.format(address_name, 
                                                               address_latitude, 
                                                               address_longitude))

Latitude and longitude values of 2825 Grandview Hwy, Vancouver are 49.258413, -123.046053.


In [36]:
# get the top 100 venues within a radius of 500 meters of 2825 Grandview Hwy, Vancouver
# create URL

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    address_latitude, 
    address_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=U5L4NWGMXHTGERAFDMMSWIG3H1VHLQIV4EIAKKVWNAO5VPRW&client_secret=WK234YY1I1THXQJO11OQH35LY4FLCJQ3Q1REU3E21JHOXW5Y&v=20180605&ll=49.258413,-123.046053&radius=500&limit=100'

In [37]:
# send the GET request

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f45a87e2df029194a3e7787'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Renfrew-Collingwood',
  'headerFullLocation': 'Renfrew-Collingwood, Vancouver',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 17,
  'suggestedBounds': {'ne': {'lat': 49.2629130045, 'lng': -123.03917088440987},
   'sw': {'lat': 49.253912995499995, 'lng': -123.05293511559013}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '541dbe0b498e353abeacd343',
       'name': 'Starbucks',
       'location': {'address': '2905 Hebb Ave, Broadway Tech Centre',
        'crossStreet': 'Renfrew',
        'lat': 49.259884,
        'lng': -123.043461,
        'labeledLatL

In [38]:
# use the get_category_type function from the Foursquare lab
# a function that extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
# clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# show nearby venues
nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,49.259884,-123.043461
1,Dario Italian Restaurant,Italian Restaurant,49.257309,-123.049924
2,Broadway Tech Centre,Office,49.260423,-123.043000
3,Tisol,Pet Store,49.258728,-123.040452
4,Lowe's,Hardware Store,49.259563,-123.047478


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


In [46]:
# create a function to repeat the same process for all the addresses in Vancouver

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Address', 
                  'Address Latitude', 
                  'Address Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [47]:
# run the above function on each address and create a new dataframe called vancouver_venues

vancouver_venues = getNearbyVenues(names=df_data['Address'],
                                   latitudes=df_data['Latitude'],
                                   longitudes=df_data['Longitude']
                                  )

2825 Grandview Hwy, Vancouver
1748 Commercial Drive, Vancouver
1599 Venables St, Vancouver
430 Railway St, Vancouver
1951 Glen Dr, Vancouver
1868 Glen Dr, Vancouver
1868 Glen Dr, Vancouver
261 East Pender Street, Vancouver
1860 Powell St, Vancouver
4737 Hastings Street, Burnaby
15 Victoria Dr, Vancouver
2906 East 2nd Avenue, Vancouver
1470 Pemberton Ave, North Vancouver
1453 Lonsdale Ave, North Vancouver
2198 Yukon St, Vancouver
1226 Frances St, Vancouver
269 East Georgia Street, Vancouver
851 Terminal Avenue, Vancouver
455 Main St, Vancouver
3701 Hastings St, Burnaby
155 1st Street East, North Vancouver
106 West 1st Street, North Vancouver
106 West 1st Street, North Vancouver
89 East Pender Street, Vancouver
1312 Kingsway, Vancouver
1047 Granville St, Vancouver
3751 West 10th Avenue, Vancouver
1610 Hamilton Avenue, North Vancouver
506 Main Street, Vancouver
2906 East 2nd Avenue, Vancouver
1650 West 2nd Avenue, Vancouver
1650 West 2nd Avenue, Vancouver
1295 Davie Street, Vancouver
1938

In [48]:
# check the size of the resulting dataframe

print(vancouver_venues.shape)
vancouver_venues.head()

(1917, 7)


,Address,Address Latitude,Address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,Starbucks,49.259884,-123.043461,Coffee Shop
1,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,Dario Italian Restaurant,49.257309,-123.049924,Italian Restaurant
2,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,Broadway Tech Centre,49.260423,-123.043000,Office
3,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,Tisol,49.258728,-123.040452,Pet Store
4,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,Lowe's,49.259563,-123.047478,Hardware Store


In [49]:
# check how many venues were returned for each address

vancouver_venues.groupby('Address').count()

,Address Latitude,Address Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Address,,,,,,
"1047 Granville St, Vancouver",180,180,180,180,180,180
"106 West 1st Street, North Vancouver",130,130,130,130,130,130
"110 1st St E, North Vancouver",65,65,65,65,65,65
"1226 Frances St, Vancouver",20,20,20,20,20,20
"1295 Davie Street, Vancouver",31,31,31,31,31,31
"1308 West Broadway, Vancouver",54,54,54,54,54,54
"1312 Kingsway, Vancouver",30,30,30,30,30,30
"1453 Lonsdale Ave, North Vancouver",10,10,10,10,10,10
"1470 Pemberton Ave, North Vancouver",4,4,4,4,4,4


In [50]:
# check number of unique categories curated from all the returned venues

print('There are {} uniques categories.'.format(len(vancouver_venues['Venue Category'].unique())))

There are 201 uniques categories.


### Analyze Each Address

In [52]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add address column back to dataframe
vancouver_onehot['Address'] = vancouver_venues['Address']

vancouver_onehot.head()

,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Hobby Shop,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trade School,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Address
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"2825 Grandview Hwy, Vancouver"
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"2825 Grandview Hwy, Vancouver"
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [53]:
# move address column to the first column
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

,Address,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Hobby Shop,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trade School,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"2825 Grandview Hwy, Vancouver",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"2825 Grandview Hwy, Vancouver",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"2825 Grandview Hwy, Vancouver",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [54]:
# examine the new dataframe size

vancouver_onehot.shape

(1917, 202)

In [55]:
# group rows by address and take mean of frequency of occurrence of each category

vancouver_grouped = vancouver_onehot.groupby('Address').mean().reset_index()
vancouver_grouped

,Address,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Hobby Shop,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Insurance Office,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trade School,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"1047 Granville St, Vancouver",0.011111,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.044444,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.011111,0.011111,0.022222,0.00,0.033333,0.000000,0.011111,0.000000,0.011111,0.000000,0.000000,0.011111,0.000000,0.011111,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.022222,0.022222,0.000000,0.000000,0.011111,0.000000,0.000000,0.000000,0.011111,0.011111,0.000000,0.00,0.000000,0.000000,0.022222,0.011111,0.011111,0.000000,0.000000,0.011111,0.000000,0.011111,0.000000,0.011111,0.055556,0.00000,0.022222,0.000000,0.011111,0.00,0.000000,0.000000,0.044444,0.011111,0.044444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.00,0.000000,0.033333,0.011111,0.000000,0.000000,

In [56]:
# confirm new size

vancouver_grouped.shape

(45, 202)

In [57]:
# print each address along with the top 5 most common venues

num_top_venues = 5

for hood in vancouver_grouped['Address']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Address'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----1047 Granville St, Vancouver----
                 venue  freq
0                Hotel  0.06
1               Bakery  0.04
2   Italian Restaurant  0.04
3  Japanese Restaurant  0.04
4           Restaurant  0.04


----106 West 1st Street, North Vancouver----
                venue  freq
0         Coffee Shop  0.08
1           Gastropub  0.06
2    Sushi Restaurant  0.05
3   Korean Restaurant  0.05
4  Frozen Yogurt Shop  0.03


----110 1st St E, North Vancouver----
                venue  freq
0         Coffee Shop  0.08
1           Gastropub  0.06
2    Sushi Restaurant  0.05
3   Korean Restaurant  0.05
4  Frozen Yogurt Shop  0.03


----1226 Frances St, Vancouver----
              venue  freq
0           Brewery  0.30
1    Ice Cream Shop  0.10
2             Hotel  0.05
3  Sushi Restaurant  0.05
4     Grocery Store  0.05


----1295 Davie Street, Vancouver----
              venue  freq
0            Bakery  0.10
1              Café  0.06
2  Greek Restaurant  0.06
3           Gay Bar  0.06
4   

                  venue  freq
0      Sushi Restaurant   0.1
1  Fast Food Restaurant   0.1
2                  Park   0.1
3           Pizza Place   0.1
4    Chinese Restaurant   0.1


----506 Main Street, Vancouver----
                venue  freq
0                Café  0.07
1      Sandwich Place  0.06
2    Asian Restaurant  0.04
3  Chinese Restaurant  0.04
4         Coffee Shop  0.04


----726 Kingsway, Vancouver----
                   venue  freq
0                 Bakery  0.11
1  Vietnamese Restaurant  0.11
2            Pizza Place  0.07
3     Mexican Restaurant  0.07
4                   Park  0.07


----851 Terminal Avenue, Vancouver----
            venue  freq
0  Hardware Store   0.1
1            Park   0.1
2        Platform   0.1
3            Café   0.1
4     Flea Market   0.1


----89 East Pender Street, Vancouver----
                venue  freq
0                Café  0.06
1         Coffee Shop  0.05
2         Pizza Place  0.04
3  Chinese Restaurant  0.04
4      Sandwich Place  0.03

### Put top 5 venues into a pandas dataframe

In [58]:
# write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
# create the new dataframe and display the top 10 venues for each address

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Address']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
address_venues_sorted = pd.DataFrame(columns=columns)
address_venues_sorted['Address'] = vancouver_grouped['Address']

for ind in np.arange(vancouver_grouped.shape[0]):
    address_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venues)

address_venues_sorted.head()

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"1047 Granville St, Vancouver",Hotel,Japanese Restaurant,Italian Restaurant,Restaurant,Bakery,Concert Hall,Mexican Restaurant,Burger Joint,Taco Place,Ice Cream Shop
1,"106 West 1st Street, North Vancouver",Coffee Shop,Gastropub,Korean Restaurant,Sushi Restaurant,Greek Restaurant,Restaurant,Café,Breakfast Spot,Convenience Store,Sandwich Place
2,"110 1st St E, North Vancouver",Coffee Shop,Gastropub,Korean Restaurant,Sushi Restaurant,Greek Restaurant,Restaurant,Café,Breakfast Spot,Convenience Store,Sandwich Place
3,"1226 Frances St, Vancouver",Brewery,Ice Cream Shop,Gourmet Shop,Hotel,Bus Line,Sushi Restaurant,Discount Store,Chinese Restaurant,Building,Coffee Shop
4,"1295 Davie Street, Vancouver",Bakery,Farmers Market,Gay Bar,Park,Café,Greek Restaurant,Food Truck,Bookstore,South Indian Restaurant,Pub


### Cluster Addresses

In [60]:
# run k-means to cluster the address into 5 clusters

# set number of clusters
kclusters = 5

vancouver_grouped_clustering = vancouver_grouped.drop('Address', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(vancouver_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 0, 4, 4, 0, 0, 3, 0], dtype=int32)

In [61]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each address

# add clustering labels
address_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

vancouver_merged = df_data

# merge vancouver_grouped with df_data to add latitude/longitude for each neighborhood
vancouver_merged = vancouver_merged.join(address_venues_sorted.set_index('Address'), on='Address')

vancouver_merged.head(25) # check the last columns!

,Address,Latitude,Longitude,SquareFeet,LeaseRate,Walkable,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"2825 Grandview Hwy, Vancouver",49.258413,-123.046053,1000,$35.00,No,4,Pet Store,Coffee Shop,Paper / Office Supplies Store,Insurance Office,Sandwich Place,Restaurant,Chinese Restaurant,Office,Italian Restaurant,Mexican Restaurant
1,"1748 Commercial Drive, Vancouver",49.268816,-123.069260,1325,$13.25,No,4,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Deli / Bodega,Indian Restaurant,Pizza Place,Park,Gastropub,Record Shop
2,"1599 Venables St, Vancouver",49.276962,-123.072991,6600,$16.00,No,4,Coffee Shop,Brewery,Indian Restaurant,Grocery Store,Pizza Place,French Restaurant,Breakfast Spot,Furniture / Home Store,Fried Chicken Joint,Bus Line
3,"430 Railway St, Vancouver",49.284616,-123.094533,8775,$18.74,No,4,Diner,Latin American Restaurant,Brewery,Boxing Gym,Park,Café,Food Truck,Italian Restaurant,Restaurant,Bakery
4,"1951 Glen Dr, Vancouver",49.267358,-123.082534,534,$25.46,No,2,Coffee Shop,Restaurant,Furniture / Home Store,Hardware Store,Monument / Landmark,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,"1868 Glen Dr, Vancouver",49.268413,-123.080809,517,$25.61,No,2,Coffee Shop,Restaurant,Furniture / Home Store,Hardware Store,Monument / Landmark,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
6,"1868 Glen Dr, Vancouver",49.268413,-123.080809,880,$25.61,No,2,Coffee Shop,Restaurant,Furniture / Home Store,Hardware Store,Monument / Landmark,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
7,"261 East Pender Street, Vancouver",49.280606,-123.098228,4200,$31.17,No,4,Café,Pizza Place,Chinese Restaurant,Sandwich Place,Noodle House,Beer Bar,Bar,Diner,Restaurant,Asian Restaurant
8,"1860 Powell St, Vancouver",49.284258,-123.065888,2030,$31.45,No,0,Café,Coffee Shop,Asian Restaurant,Bus Stop,Harbor / Marina,Chinese Restaurant,Brewery,Sandwich Place,Middle Eastern Restaurant,Bus Station
9,"4737 Hastings Street, Burnaby",49.281256,-122.995821,1522,$31.53,No,4,Park,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Sushi Restaurant,Japanese Restaurant,Dessert Shop,Coffee Shop,Chocolate Shop,Martial Arts School


In [62]:
# check number of rows in dataframe
vancouver_merged.shape

(51, 17)

In [64]:
# count NaN in Cluster Labels column
vancouver_merged['Cluster Labels'].isna().sum()

0

In [65]:
# visualize the resulting clusters

latitude = 49.280000
longitude = -123.046053

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='Stamen Terrain')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(vancouver_merged['Latitude'], vancouver_merged['Longitude'], vancouver_merged['Address'], vancouver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [68]:
# Cluster 1

vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 0, vancouver_merged.columns[[0] + list(range(3, vancouver_merged.shape[1]))]]

,Address,SquareFeet,LeaseRate,Walkable,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"1860 Powell St, Vancouver",2030,$31.45,No,0,Café,Coffee Shop,Asian Restaurant,Bus Stop,Harbor / Marina,Chinese Restaurant,Brewery,Sandwich Place,Middle Eastern Restaurant,Bus Station
10,"15 Victoria Dr, Vancouver",2163,$33.45,No,0,Café,Bus Stop,Brewery,Chinese Restaurant,Harbor / Marina,Coffee Shop,Vietnamese Restaurant,Asian Restaurant,Bus Station,Japanese Restaurant
13,"1453 Lonsdale Ave, North Vancouver",4150,$23.00,No,0,Coffee Shop,Convenience Store,Gym / Fitness Center,Donut Shop,Sandwich Place,Fast Food Restaurant,Grocery Store,Bar,Burger Joint,Harbor / Marina
15,"1226 Frances St, Vancouver",3116,$23.30,No,0,Brewery,Ice Cream Shop,Gourmet Shop,Hotel,Bus Line,Sushi Restaurant,Discount Store,Chinese Restaurant,Building,Coffee Shop
24,"1312 Kingsway, Vancouver",950,$30.00,No,0,Vietnamese Restaurant,Chinese Restaurant,Vegetarian / Vegan Restaurant,Indian Restaurant,Escape Room,Seafood Restaurant,Sandwich Place,Park,Market,Insurance Office
44,"2331 Main Street, Vancouver",2060,$35.00,No,0,Coffee Shop,Brewery,Diner,Bakery,Sushi Restaurant,Breakfast Spot,Sandwich Place,Café,Indian Restaurant,Chinese Restaurant


In [69]:
# Cluster 2

vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 1, vancouver_merged.columns[[0] + list(range(3, vancouver_merged.shape[1]))]]

,Address,SquareFeet,LeaseRate,Walkable,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,"1610 Hamilton Avenue, North Vancouver",1211,$30.70,Yes,1,Park,Yoga Studio,Electronics Store,Food & Drink Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [70]:
# Cluster 3

vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 2, vancouver_merged.columns[[0] + list(range(3, vancouver_merged.shape[1]))]]

,Address,SquareFeet,LeaseRate,Walkable,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"1951 Glen Dr, Vancouver",534,$25.46,No,2,Coffee Shop,Restaurant,Furniture / Home Store,Hardware Store,Monument / Landmark,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,"1868 Glen Dr, Vancouver",517,$25.61,No,2,Coffee Shop,Restaurant,Furniture / Home Store,Hardware Store,Monument / Landmark,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
6,"1868 Glen Dr, Vancouver",880,$25.61,No,2,Coffee Shop,Restaurant,Furniture / Home Store,Hardware Store,Monument / Landmark,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
35,"2568 Arbutus Street, Vancouver",1300,$32.31,No,2,Coffee Shop,Yoga Studio,Grocery Store,Pharmacy,Park,Restaurant,Café,Business Service,Pet Store,Vegetarian / Vegan Restaurant


In [71]:
# Cluster 4

vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 3, vancouver_merged.columns[[0] + list(range(3, vancouver_merged.shape[1]))]]

,Address,SquareFeet,LeaseRate,Walkable,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"1470 Pemberton Ave, North Vancouver",6764,$20.94,Yes,3,Convenience Store,Bus Stop,Park,Golf Course,Yoga Studio,Escape Room,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


In [72]:
# Cluster 5

vancouver_merged.loc[vancouver_merged['Cluster Labels'] == 4, vancouver_merged.columns[[0] + list(range(3, vancouver_merged.shape[1]))]]

,Address,SquareFeet,LeaseRate,Walkable,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"2825 Grandview Hwy, Vancouver",1000,$35.00,No,4,Pet Store,Coffee Shop,Paper / Office Supplies Store,Insurance Office,Sandwich Place,Restaurant,Chinese Restaurant,Office,Italian Restaurant,Mexican Restaurant
1,"1748 Commercial Drive, Vancouver",1325,$13.25,No,4,Coffee Shop,Italian Restaurant,Japanese Restaurant,Sushi Restaurant,Deli / Bodega,Indian Restaurant,Pizza Place,Park,Gastropub,Record Shop
2,"1599 Venables St, Vancouver",6600,$16.00,No,4,Coffee Shop,Brewery,Indian Restaurant,Grocery Store,Pizza Place,French Restaurant,Breakfast Spot,Furniture / Home Store,Fried Chicken Joint,Bus Line
3,"430 Railway St, Vancouver",8775,$18.74,No,4,Diner,Latin American Restaurant,Brewery,Boxing Gym,Park,Café,Food Truck,Italian Restaurant,Restaurant,Bakery
7,"261 East Pender Street, Vancouver",4200,$31.17,No,4,Café,Pizza Place,Chinese Restaurant,Sandwich Place,Noodle House,Beer Bar,Bar,Diner,Restaurant,Asian Restaurant
9,"4737 Hastings Street, Burnaby",1522,$31.53,No,4,Park,Pizza Place,Chinese Restaurant,Fast Food Restaurant,Sushi Restaurant,Japanese Restaurant,Dessert Shop,Coffee Shop,Chocolate Shop,Martial Arts School
11,"2906 East 2nd Avenue, Vancouver",820,$20.00,No,4,Chinese Restaurant,Noodle House,Seafood Restaurant,Tea Room,Italian Restaurant,Thrift / Vintage Store,Korean Restaurant,Breakfast Spot,Liquor Store,Park
14,"2198 Yukon St, Vancouver",6938,$23.00,No,4,Coffee Shop,Café,Japanese Restaurant,Ramen Restaurant,Sushi Restaurant,Bagel Shop,Mexican Restaurant,Italian Restaurant,Grocery Store,Art Gallery
16,"269 East Georgia Street, Vancouver",2930,$24.00,No,4,Sandwich Place,Café,Chinese Restaurant,Bar,Asian Restaurant,Noodle House,Pizza Place,Lounge,Vietnamese Restaurant,Gourmet Shop
17,"851 Terminal Avenue, Vancouver",15451,$24.76,No,4,Flea Market,Auto Dealership,Furniture / Home Store,Climbing Gym,Café,Coffee Shop,Hardware Store,Pet Store,Platform,Park
